In [0]:
from pyspark.sql import SparkSession
from pyspark.sql import functions as F
import pandas as pd
import numpy as np

# Cria SparkSession
spark = SparkSession.builder.getOrCreate()

# Leitura do DataFrame
df = spark.table("projeto_final_silver.silver_panorama_eja")
# Conversão pra Dataframe do pandas
df = df.toPandas()

df.head()

### Criando coluna com total da população por faixa-etária

In [0]:
%sql
DESCRIBE projeto_final_silver.silver_panorama_eja

In [0]:
df['total_15_19'] = df['15_19_anos_populacao_nao_alfabetizada'] * df['populacao_nao_alfabetizada_censo_ibge_2022'] / 100
df['total_20_24'] = df['20_24_anos_populacao_nao_alfabetizada'] * df['populacao_nao_alfabetizada_censo_ibge_2022'] / 100
df['total_25_34'] = df['25_34_anos_populacao_nao_alfabetizada'] * df['populacao_nao_alfabetizada_censo_ibge_2022'] / 100
df['total_35_44'] = df['35_44_anos_populacao_nao_alfabetizada'] * df['populacao_nao_alfabetizada_censo_ibge_2022'] / 100
df['total_45_54'] = df['45_54_anos_populacao_nao_alfabetizada'] * df['populacao_nao_alfabetizada_censo_ibge_2022'] / 100
df['total_55_64'] = df['55_64_anos_populacao_nao_alfabetizada'] * df['populacao_nao_alfabetizada_censo_ibge_2022'] / 100
df['total_65_mais'] = df['65_anos_ou_mais_populacao_nao_alfabetizada'] * df['populacao_nao_alfabetizada_censo_ibge_2022'] / 100

In [0]:
df['percentual_15_19'] = df['15_19_anos_populacao_nao_alfabetizada'] / df['populacao_nao_alfabetizada_censo_ibge_2022'] 
df['percentual_20_24'] = df['20_24_anos_populacao_nao_alfabetizada'] / df['populacao_nao_alfabetizada_censo_ibge_2022']  
df['percentual_25_34'] = df['25_34_anos_populacao_nao_alfabetizada'] / df['populacao_nao_alfabetizada_censo_ibge_2022']  
df['percentual_35_44'] = df['35_44_anos_populacao_nao_alfabetizada'] / df['populacao_nao_alfabetizada_censo_ibge_2022']
df['percentual_45_54'] = df['45_54_anos_populacao_nao_alfabetizada'] / df['populacao_nao_alfabetizada_censo_ibge_2022'] 
df['percentual_55_64'] = df['55_64_anos_populacao_nao_alfabetizada'] / df['populacao_nao_alfabetizada_censo_ibge_2022']
df['percentual_65_mais'] = df['65_anos_ou_mais_populacao_nao_alfabetizada'] / df['populacao_nao_alfabetizada_censo_ibge_2022'] 

### coluna aderiu ao pacto sim ou não

In [0]:
df['aderiu_pacto'] = np.where(df['ano_adesao'].isin(['2024', '2025']),'sim','nao')

### criando coluna para docentes de outros níveis

In [0]:
df['docentes_outros_niveis'] = (df['total_geral_docentes_ed_basica'] - df['total_geral_docentes_ens_fund']).fillna(0).astype(int)

### persistindo os dados na gold

In [0]:
spark.sql("DROP TABLE IF EXISTS projeto_final_gold.gold_panorama_eja")

In [0]:
df_spark = spark.createDataFrame(df)

# Salva como tabela Delta no schema Gold
df_spark.write.format("delta").mode("overwrite").saveAsTable("projeto_final_gold.gold_panorama_eja")